## Build a model from python

In [ ]:
import hydromt
from hydromt_sfincs import SfincsModel, utils
import geopandas as gpd
import pandas as pd
import xarray as xr

This example shows you how to setup a model schematization from existing preprocessed GIS raster data and manually add forcing data. Note that all rasters should be on exactly the same grid and must have proper metadata including the CRS which must be a projected system with unit in meters, such as a UTM zone!

To understand how the data is stored in the SfincsModel object we have a look at some internal variables:

- MAPS: saved as `hydromt.RasterDataset`; use `.set_staticmaps()` to add new maps
- GEOMS: saved as `dict` of `geopandas.GeoDataFrame`; use `.set_staticgeoms()` to add new geometries
- FORCING: saved as `dict` of `xarray.DataArray` or `hydromt.GeoDataArray` in case the forcing is for specific xy locations; use `.set_forcing()` in general, or the convenience `.set_forcing_1d` for 1D forcing.

The following MAPS, GEOMS and FORCINGS are currently recognized:

In [ ]:
SfincsModel._MAPS.values()

In [ ]:
SfincsModel._GEOMS.values()

In [ ]:
SfincsModel._FORCING.values()

### read raster data

Here we first make a directory with an elevation and mask raster files from which we can build the model.
For the sake of this example we export these layers from an existing model. 
In practice you could setup these layers in a GIS such as QGIS based on your own data.

In [ ]:
# export a model to gis files using hydromt (only for sake of the example)
mod0 = SfincsModel(root="sfincs_riverine", mode="r")
mod0.read()
mod0.write_raster(variables=["staticmaps.dep"], root="gis_data")

In [ ]:
# check content of our folder
!ls gis_data

In [ ]:
# check meta data on our geotiff file (specifically note the crs attribute!)
!rio info gis_data/dep.tif

Here we read a single raster (dep.tif) using the hydromt.open_raster method. If all data in the a folder is on an identical grid we can read it into a single xarray.Dataset with the hydromt.open_mfraster method.

In [ ]:
# read all data to a Dataset
# note that the names are taken from the file basenames
da_dep = hydromt.open_raster("gis_data/dep.tif")
da_dep

### setup SFINCS model

First we initiate an new model instance in a new model root folder in writing mode:

In [ ]:
mod = SfincsModel(root="sfincs_from_py", mode="w")
# Note this is still an empty model with no maps
len(mod.staticmaps) == 0

We can add staticmaps to the model using the `set_staticmaps()` method. Here it is important that the model layers are on a identical grid, otherwise these will not be excepted. 

Note that geotiff raster typically has a N->S orientation while the SFINCS model uses a S->N orientation. If the staticmaps are  kept in N->S orientation the y-axis will be flipped by the `set_staticmaps()` method. Compare the y dimension between ds and .staticmaps to check.

In [ ]:
mod.set_staticmaps(da_dep, name="dep")
mod.staticmaps

Next we load a default config which we need to modify for our example

In [ ]:
mod.read_config()  ## laods default config if no sfincs.inp is found in model root
mod.update_spatial_attrs()  # sets mmax, nmax, dx, dy, x0, y0 & epsg based on staticmaps
mod.config  # inspect config

Next we need to define the mask and boundaries where water can leave the model domain. For a riverine application this can be done using the ``setup_river_outflow()`` method to find set cells boundary cells within a range from river outflow points to an open boundary. The more general ``setup_bounds()`` method can be used to setup both open and water level boundary cells.

In [ ]:
# setup mask with open boundaries at the downstream end of
mod.setup_mask()  # sets mask based on valid dep cells
mod.setup_river_hydrography()  # create hydrography data from dep file
mod.setup_river_outflow()  # use default settings

In [ ]:
# let's visualize the model schematization
_ = mod.plot_basemap(fn_out=None, vmax=500)

### Add discharge forcing

The SFINCS discharge forcing is set based on a combination of a the src (xy point locations) and dis (timeseries) files. In hydroMT the src locations are represented by a geopandas.GeoDataFrame in the staticgeoms attribute and dis timeseries by a xarray.DataArray with 'time' and 'index' dimensions in the forcing attribute. SFINCS waterlevel forcing can be set similarly using 'bzd' (xy point locations) and 'bzs' (timeseries) and precipitation with 'precip' (timeseries). 

NOTE: If csv or netcdf files with location and timeseries are available, the `setup_q_forcing()` method is recommended. River inflow points can also be found based on a flow direction dataset using `setup_river_inflow()`

In [ ]:
# here we create three source points with a simple triangular discharge hydrograph
# setup locations
x = [264891.02, 264903.71, 277443.57]
y = [5083000.61, 5085039.90, 5091621.70]
pnts = gpd.points_from_xy(x, y)
index = [1, 2, 3]  # NOTE that the index should start at one
src = gpd.GeoDataFrame(index=index, geometry=pnts, crs=mod.crs)
src

In [ ]:
# setup discharge timeseries
mod.set_config("tref", "20210101 000000")
mod.set_config("tstart", "20210101 000000")
mod.set_config("tstop", "20210102 000000")
time = pd.date_range(
    start=utils.parse_datetime(mod.config["tstart"]),
    end=utils.parse_datetime(mod.config["tstop"]),
    periods=3,
)
ts = pd.DataFrame(
    index=time, columns=index, data=[[100, 50, 80], [500, 250, 300], [100, 50, 80]]
)
ts

In [ ]:
# update forcing in model
mod.set_forcing_1d(name="discharge", ts=ts, xy=src)

In [ ]:
# Note that the data has been converted to a GeoDataset which combines the timeseries and spatial data.
mod.forcing["dis"]

In [ ]:
# to retrieve the src point locations use the `.vector.to_gdf()` method
mod.forcing["dis"].vector.to_gdf()

In [ ]:
# The config has also been updated with a disfile and srcfile entry
mod.config

In [ ]:
# let's visualize the model forcing timeseries
# TIP: rerun the plot_basemaps above to see the src point locations
_ = mod.plot_forcing(fn_out=None)

### write to file

If we are satisfied with the model schematization and forcing we can write it to disk. 
Here we use the `write` method to write the full model schematization, seperate components can be writen individually with write_<component> methods e.g. `write_staticmaps`, `write_config` etc.

In [ ]:
mod.write()  #